In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression as LR
import os
from scipy.stats import norm

In [2]:
athlete_data = pd.read_csv("2025_Problem_C_Data/summerOly_athletes.csv")
athlete_data["Medal"] = athlete_data["Medal"].map({'No medal' : 0, 'Bronze' : 1, 'Silver' : 2, 'Gold' : 3})
athlete_data["Sex"] = athlete_data["Sex"].map({'M' : 1, 'F' : 0})
athlete_data.head(6)

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
0,A Dijiang,1,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,0
1,A Lamusi,1,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,0
2,Gunnar Aaby,1,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,0
3,Edgar Aabye,1,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,3
4,Cornelia (-strannood),0,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,0
5,Cornelia (-strannood),0,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 4 x 100 metres Relay,0


### Tables 
The following defines functions that will used the parced data from the athlete_data to create tables of the medal counts in specific sports.

In [3]:
matrix = athlete_data.to_numpy()
from collections import defaultdict
b = defaultdict(set)
si = defaultdict(set)
g = defaultdict(set)
sports = set()
country_years = defaultdict(set)
country_code = dict()
for row in matrix:
    country_code[row[2]] = row[3]
    i = row[3], row[4], row[6]
    sports.add(row[6])
    country_years[row[3]].add(row[4])
    if row[-1] == 1: b[i].add(row[-2])
    if row[-1] == 2: si[i].add(row[-2])
    if row[-1] == 3: g[i].add(row[-2])
years = [1896,1900,1904,1906,1908,1912,1920,1924,1928,1932,1936,1948,1952,1956,1960,1964,1968,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024]
def con_years(country):
    year = list()
    for i in years:
        if i in country_years[country]: year.append(i)
    return year
def country_total(country):
    years = con_years(country)
    data = np.zeros((len(sports) + 1, len(years) + 2), dtype = object)
    i = 1
    for s in sports: 
        data[i, 0] = s
        i += 1
    data[0,1:-1] = years
    for r in range(1,len(data)):
        for c in range(1,len(data[0]) - 1):
            data[r, c] = len(b[(country, data[0, c], data[r, 0])]) + len(si[(country, data[0, c], data[r, 0])]) + len(g[(country, data[0, c], data[r, 0])])
    data[1:,-1] = [np.sum(row) for row in data[1:,1:-1]]
    data[0,-1] = np.sum(data[:,-1])
    return data

def country_bronze(country):
    years = con_years(country)
    data = np.zeros((len(sports) + 1, len(years) + 2), dtype = object)
    i = 1
    for s in sports: 
        data[i, 0] = s
        i += 1
    data[0,1:-1] = years
    for r in range(1,len(data)):
        for c in range(1,len(data[0]) - 1):
            data[r, c] = len(b[(country, data[0, c], data[r, 0])])
    data[1:,-1] = [np.sum(row) for row in data[1:,1:-1]]
    data[0,-1] = np.sum(data[:,-1])
    return data

def country_silver(country):
    years = con_years(country)
    data = np.zeros((len(sports) + 1, len(years) + 2), dtype = object)
    i = 1
    for s in sports: 
        data[i, 0] = s
        i += 1
    data[0,1:-1] = years
    for r in range(1,len(data)):
        for c in range(1,len(data[0]) - 1):
            data[r, c] = len(si[(country, data[0, c], data[r, 0])])
    data[1:,-1] = [np.sum(row) for row in data[1:,1:-1]]
    data[0,-1] = np.sum(data[:,-1])
    return data

def country_gold(country):
    years = con_years(country)
    data = np.zeros((len(sports) + 1, len(years) + 2), dtype = object)
    i = 1
    for s in sports: 
        data[i, 0] = s
        i += 1
    data[0,1:-1] = years
    for r in range(1,len(data)):
        for c in range(1,len(data[0]) - 1):
            data[r, c] = len(g[(country, data[0, c], data[r, 0])])
    data[1:,-1] = [np.sum(row) for row in data[1:,1:-1]]
    data[0,-1] = np.sum(data[:,-1])
    return data


### Get Files
The following turns the tables into files and exludes any that did not have any medals

In [75]:
no_medal = list()
for c in country_years:
    total = country_total(c)
    if total[0,-1] == 0: 
        no_medal.append(c)
        continue
    os.makedirs('2025_Problem_C_Data/extraExtra/' + c)
    frame = pd.DataFrame(total)
    frame.to_csv('2025_Problem_C_Data/extraExtra/' + c + '/total.csv', index=False)
    bronze = country_bronze(c)
    if bronze[0,-1] != 0:
        frame = pd.DataFrame(bronze)
        frame.to_csv('2025_Problem_C_Data/extraExtra/' + c + '/bronze.csv', index=False)
    silver = country_silver(c)
    if silver[0,-1] != 0:
        frame = pd.DataFrame(silver)
        frame.to_csv('2025_Problem_C_Data/extraExtra/' + c + '/silver.csv', index=False)
    gold = country_gold(c)
    if gold[0,-1] != 0:
        frame = pd.DataFrame(gold)
        frame.to_csv('2025_Problem_C_Data/extraExtra/' + c + '/gold.csv', index=False)
print(no_medal)

['CHA', 'NCA', 'LBA', 'PLE', 'COM', 'BRU', 'MDV', 'YAR', 'CGO', 'BEN', 'SOM', 'MLI', 'ANG', 'BAN', 'ESA', 'HON', 'SEY', 'MTN', 'SKN', 'VIN', 'LBR', 'NEP', 'PLW', 'ASA', 'SAM', 'RWA', 'MLT', 'GUI', 'BIZ', 'YMD', 'SLE', 'PNG', 'YEM', 'OMA', 'VAN', 'IVB', 'CAF', 'MAD', 'MAL', 'BIH', 'GUM', 'CAY', 'GBS', 'TLS', 'COD', 'LAO', 'ROT', 'CAM', 'SOL', 'CRT', 'GEQ', 'BOL', 'SAA', 'ANT', 'AND', 'FSM', 'MYA', 'MAW', 'RHO', 'STP', 'LIE', 'GAM', 'COK', 'SWZ', 'NBO', 'ARU', 'NRU', 'VNM', 'BHU', 'MHL', 'KIR', 'UNK', 'TUV', 'NFL', 'SSD', 'LES', 'LBN']


The Following creates the files with the tables that dont have medals.

In [4]:
for c in country_years:
    total = country_total(c)
    os.makedirs('2025_Problem_C_Data/extraExtraFull/' + c)
    frame = pd.DataFrame(total)
    frame.to_csv('2025_Problem_C_Data/extraExtraFull/' + c + '/total.csv', index=False)
    bronze = country_bronze(c)
    frame = pd.DataFrame(bronze)
    frame.to_csv('2025_Problem_C_Data/extraExtraFull/' + c + '/bronze.csv', index=False)
    silver = country_silver(c)
    frame = pd.DataFrame(silver)
    frame.to_csv('2025_Problem_C_Data/extraExtraFull/' + c + '/silver.csv', index=False)
    gold = country_gold(c)
    frame = pd.DataFrame(gold)
    frame.to_csv('2025_Problem_C_Data/extraExtraFull/' + c + '/gold.csv', index=False)

### Line of best fit
The following is a function that make a line of best fit with weights.

In [9]:
def lbf_weighted(x, y, weights):
    w = int(np.sum(weights))
    wx = int(np.dot(weights, x))
    wy = int(np.dot(weights, y))
    wxx = int(np.dot(weights, x * x))
    wxy = int(np.dot(weights, x * y))
    slope = (w * wxy - wx * wy) / (w * wxx - wx**2)
    return slope, (wy - slope * wx) / w

This fits the linear model to all the sports of a country

In [10]:
def fit_linearW(country_data, weights):
    data = np.zeros((len(country_data), 4), dtype = object)
    for row in range(1,len(country_data)):
        data[row,0] = country_data[row,0]
        medals = country_data[row, 1:-1]
        years = country_data[0, 1:-1]
        data[row, 1:3] = lbf_weighted(years, medals, weights)
        data[0,3] = country_data[0,-2] + 4
        data[row, 3] = data[row, 1]*(data[0,3]) + data[row, 2]
        data[0,0] = np.sum(data[1:,3])
        diff = country_data[1:,-1] - data[1:,-1]
        error = np.dot(diff, diff)/ len(diff)
        data[0,1] = error
    return data

This calculates the models prediction for a specific year and a series of countries for a year.

In [11]:
def predict(country, country_med, f = lambda x: x*x, year = 2028):
    lol = (year-2028)// 4
    if year == 2028: lol = 100000
    tab = country_med[:,:lol]
    w = np.array([f(i) for i in range(len(tab[0])-2)])
    mod = fit_linearW(tab, w)
    return mod[0, 0]
def table(countries, year):
    data = np.zeros((len(countries) + 1, 9), dtype = object)
    data[0, 1:9:2] = 'Gold', 'Silver', 'Bronze', 'Total'
    data[1:,0] = countries
    lol = (year-2028)// 4
    if year == 2028: lol = 100000
    for row in range(0, len(countries)):
        tab = country_gold(countries[row])[:,:lol]
        w = np.array([i**2 for i in range(len(tab[0])-2)])
        mod = fit_linearW(tab, w)
        data[row + 1, 1:3] = mod[0, 0], mod[0, 1]
        
        tab = country_silver(countries[row])[:,:lol]
        w = np.array([i**2 for i in range(len(tab[0])-2)])
        mod = fit_linearW(tab, w)
        data[row + 1, 3:5] = mod[0, 0], mod[0, 1]

        tab = country_bronze(countries[row])[:,:lol]
        w = np.array([i**2 for i in range(len(tab[0])-2)])
        mod = fit_linearW(tab, w)
        data[row + 1, 5:7] = mod[0, 0], mod[0, 1]

        tab = country_total(countries[row])[:,:lol]
        w = np.array([i**2 for i in range(len(tab[0])-2)])
        mod = fit_linearW(tab, w)
        data[row + 1, 7:9] = mod[0, 0], mod[0, 1]
    return data

these are the 2024 and 2028 predictions for our models along with the averge error squared for each model fitted to the sports. (Note the 2028 has nonsense numbers as the error, as that data is unavalible)

In [12]:
wowza = table(['USA', 'CHN', 'JPN', 'AUS', 'FRA', 'GBR', 'KOR', 'ITA', 'GER', 'NZL'], 2024)
frame = pd.DataFrame(wowza)
frame.head(11)

,0,1,2,3,4,5,6,7,8
0,0,Gold,0,Silver,0,Bronze,0,Total,0
1,USA,42.940702,1.321193,38.371388,0.491496,34.583675,0.539474,115.895764,3.628055
2,CHN,41.398488,0.449752,28.696742,0.449002,24.854093,0.603237,94.949323,1.694124
3,JPN,15.21933,0.550091,11.279432,0.113813,16.701585,0.300132,43.200348,1.398703
4,AUS,13.580022,0.296303,14.599309,0.273724,18.298622,0.451524,46.477953,1.370962
5,FRA,10.725925,0.297765,13.284872,0.348963,14.523511,0.321699,38.534308,1.347117
6,GBR,20.805366,0.422431,16.459905,0.425766,17.335907,0.742381,54.601179,2.698763
7,KOR,10.755092,0.082929,6.971658,0.063157,10.16491,0.191675,27.89166,0.309961
8,ITA,9.394495,0.18876,9.720844,0.117839,13.070371,0.216274,32.18571,0.440042
9,GER,15.018313,0.452375,13.66436,0.245636,18.425777,0.236479,47.10845,1.759145


In [209]:
table2028 = table(['USA', 'CHN', 'JPN', 'AUS', 'FRA', 'GBR', 'KOR', 'ITA', 'GER', 'NZL'], 2028)
table2028 = table2028
frame = pd.DataFrame(wowza)
frame.head(11)

,0,1,2,3,4,5,6,7,8
0,0,Gold,0,Silver,0,Bronze,0,Total,0
1,USA,42.252814,2699.228468,39.847476,1560.270764,36.796875,1019.423818,118.897165,15153.144271
2,CHN,43.178463,91.752242,29.589221,33.634613,25.801141,26.025769,98.568826,381.703784
3,JPN,17.375706,69.609574,11.902858,36.325638,16.330715,42.813432,45.609279,415.091331
4,AUS,15.324032,78.388674,16.07701,87.279308,18.471032,93.525084,49.872074,768.904736
5,FRA,12.315419,71.560272,16.895158,71.657272,16.783896,78.493861,45.994473,633.390323
6,GBR,19.972846,103.194826,18.395698,164.199333,20.548902,123.003908,58.917445,1118.516585
7,KOR,12.043211,16.992492,7.731401,10.562421,10.501502,13.452917,30.276115,103.634509
8,ITA,10.108323,70.891935,10.220062,51.747927,13.419653,46.407234,33.748038,476.783101
9,GER,14.063536,55.84659,13.054665,69.399024,15.760265,80.632046,42.878466,570.422811


In [ ]:
frame.to_csv('2025_Problem_C_Data/table.csv', index=False)

In [16]:
counts = pd.read_csv("2025_Problem_C_Data/summerOly_medal_counts.csv")
counts['NOC'] = counts['NOC'].map(country_code)
co = counts.to_numpy()
print(co)
counts.head()

[[1 'USA' 11 ... 2 20 1896]
 [2 'GRE' 10 ... 19 47 1896]
 [3 'GER' 6 ... 2 13 1896]
 ...
 [84 'SGP' 0 ... 1 1 2024]
 [84 'SVK' 0 ... 1 1 2024]
 [84 'ZAM' 0 ... 1 1 2024]]


,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,USA,11,7,2,20,1896
1,2,GRE,10,18,19,47,1896
2,3,GER,6,5,2,13,1896
3,4,FRA,5,4,2,11,1896
4,5,GBR,2,3,2,7,1896


In [17]:
def celc_error(country, n, country_med = country_total, f = lambda x: x*x):
    if country_med == country_total: medal = 5
    elif country_med == country_bronze: medal = 4
    elif country_med == country_silver: medal = 3
    else: medal = 2
    dat = country_med(country)
    true_meds = co[co[:,1] == country]
    error = np.zeros(n)
    #for i in range(len(dat)-1-n, len(dat)-1):
    for i in range(n):
        j = i-n-1
        year = dat[0, j]
        pre = predict(country, dat, f, year)
        true_val = true_meds[true_meds[:,-1] == year][0,medal]
        error[i] = true_val - pre
    avg_error = np.sqrt(np.dot(error,error)/len(error))
    return avg_error, error
    
def celc_error_mean(country, n, country_med = country_total, f = lambda x: x*x):
    if country_med == country_total: medal = 5
    elif country_med == country_bronze: medal = 4
    elif country_med == country_silver: medal = 3
    else: medal = 2
    dat = country_med(country)
    true_meds = co[co[:,1] == country]
    error = np.zeros(n)
    #for i in range(len(dat)-1-n, len(dat)-1):
    for i in range(n):
        j = i-n-1
        year = dat[0, j]
        pre = predict(country, dat, f, year)
        true_val = true_meds[true_meds[:,-1] == year][0,medal]
        error[i] = pre - true_val
    avg_error = np.sum((error))/len(error)
    return avg_error, error

The following code finds the error of our model

In [14]:
def celc_errorp(country, n, country_med = country_total, f = lambda x: x*x):
    if country_med == country_total: medal = 5
    elif country_med == country_bronze: medal = 4
    elif country_med == country_silver: medal = 3
    else: medal = 2
    dat = country_med(country)
    true_meds = co[co[:,1] == country]
    error = np.zeros(n)
    #for i in range(len(dat)-1-n, len(dat)-1):
    for i in range(n):
        j = i-n-1
        year = dat[0, j]
        pre = predict(country, dat, f, year)
        true_val = true_meds[true_meds[:,-1] == year][0,medal]
        error[i] = (pre - true_val)/true_val
    avg_error = np.sqrt(np.dot(error,error)/len(error))
    return avg_error, error
    
def celc_error_meanp(country, n, country_med = country_total, f = lambda x: x*x):
    if country_med == country_total: medal = 5
    elif country_med == country_bronze: medal = 4
    elif country_med == country_silver: medal = 3
    else: medal = 2
    dat = country_med(country)
    true_meds = co[co[:,1] == country]
    error = np.zeros(n)
    #for i in range(len(dat)-1-n, len(dat)-1):
    for i in range(n):
        j = i-n-1
        year = dat[0, j]
        pre = predict(country, dat, f, year)
        true_val = true_meds[true_meds[:,-1] == year][0,medal]
        error[i] = (pre - true_val)/true_val
    avg_error = np.sum((error))/len(error)
    return avg_error, error

Calculate the error for USA and China

In [18]:

sd, es = celc_error('USA', 5, country_total, lambda x: x*x)
print(sd)
print(es)
mean, es = celc_error_mean('USA', 5, country_total, lambda x: x*x)
print(mean)
print(es)

7.355218417336626
[ 1.86976501 -7.61847977 10.17789463 -1.80936869 10.10423577]
-2.5448093890769456
[ -1.86976501   7.61847977 -10.17789463   1.80936869 -10.10423577]


In [19]:
sdp, es = celc_errorp('CHN', 5, country_total, lambda x: x*x)
print(sdp)
print(es)
meanp, es = celc_error_meanp('CHN', 5, country_total, lambda x: x*x)
print(meanp)
print(es)

0.22325229934751462
[-0.33421996 -0.038596    0.36581049  0.01773447  0.04339916]
0.01082563157724104
[-0.33421996 -0.038596    0.36581049  0.01773447  0.04339916]


In [20]:
p = 0.05 #p percent of cases will not fall in the error range I am calculating
z_score = norm.ppf(1 - p / 2)
low = meanp - sdp*z_score
high = meanp + sdp*z_score
print((1 - p)*100, "% of results will have a % error between ", low, " and ", high, sep = "")

95.0% of results will have a % error between -0.4267408346096426 and 0.4483920977641247


In [22]:
sdp, es = celc_errorp('USA', 5, country_total, lambda x: x*x)
print(sdp)
print(es)
meanp, es = celc_error_meanp('USA', 5, country_total, lambda x: x*x)
print(meanp)
print(es)

0.06230159587682702
[-0.01669433  0.07325461 -0.08411483  0.01601211 -0.08019235]
-0.01834695679320829
[-0.01669433  0.07325461 -0.08411483  0.01601211 -0.08019235]


In [23]:
p = 0.05 #p percent of cases will not fall in the error range I am calculating
z_score = norm.ppf(1 - p / 2)
low = meanp - sdp*z_score
high = meanp + sdp*z_score
print((1 - p)*100, "% of results will have a % error between ", low, " and ", high, sep = "")

95.0% of results will have a % error between -0.14045584089115837 and 0.1037619273047418
